In [1]:
import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
df_nutrients = pd.read_csv("rdi.csv")

In [3]:
df_nutrients

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),...,Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
0,2500.0,96.0,281.25,97.222222,30.0,11.111111,1.388889,0.125,0.125,1000.0,...,70.0,110.0,1.0467,1.6,16.7472,5.0,1.7,330.0,4.0,750.0


In [4]:
nutrients = [(key,value) for key,value in df_nutrients.to_dict("records")[0].items()]
nutrients

[('Energy (kcal/d)', 2500.0),
 ('Protein (g/d)', 96.0),
 ('Carbohydrate (g/d)', 281.25),
 ('Fat (g/d)', 97.22222222222224),
 ('Fibres (g/d)', 30.0),
 ('LA (g/d)', 11.11111111111111),
 ('ALA (g/d)', 1.3888888888888888),
 ('EPA (g/d)', 0.125),
 ('DHA (g/d)', 0.125),
 ('Calcium (mg/d)', 1000.0),
 ('Chloride (mg/d)', 3100.0),
 ('Copper (mg/d)', 1.6),
 ('Iron (mg/d)', 11.0),
 ('Iodine (µg/d)', 150.0),
 ('Magnesium (mg/d)', 350.0),
 ('Manganese (mg/d)', 3.0),
 ('Phosphorus (mg/d)', 550.0),
 ('Potassium (mg/d)', 3500.0),
 ('Selenium (µg/d)', 70.0),
 ('Sodium (mg/d)', 2000.0),
 ('Zinc (mg/d)', 10.0),
 ('Vitamin D (µg/d)', 15.0),
 ('Vitamin E (mg/d)', 13.0),
 ('Vitamin K1 (µg/d)', 70.0),
 ('Vitamin C (mg/d)', 110.0),
 ('Vitamin B1 or Thiamin (mg/d)', 1.0467000000000002),
 ('Vitamin B2 or Riboflavin (mg/d)', 1.6),
 ('Vitamin B3 or Niacin (mg/d)', 16.747200000000003),
 ('Vitamin B5 or Pantothenic acid (mg/d)', 5.0),
 ('Vitamin B6 (mg/d)', 1.7),
 ('Vitamin B9 or Folate (µg/d)', 330.0),
 ('Vitamin 

In [5]:
df_foods=pd.read_csv("ciqual_2020.csv")

In [6]:
# https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [7]:
# remove certain foods

remove_foods = ["Acerola. pulp. raw. sampled in the island of La Martiniqu", 
                "Egg. powd", "Milk. powder. semi-skimmed", 
                "Decaffeinated coffee. powder. instan",
                "Decaffeinated not instant coffee. without sugar. ready-to-drink",
                "Espresso coffee. not instant coffee. without sugar. ready-to-drink",
                "Not instant coffee. without sugar. ready-to-drink",
                "Tea. brewed. without sug",
                "Royal jelly", 
                "Cocoa powder for baby beverag", 
                "Egg white. powd", 
                "Milk. powder. skimmed", 
                "Instant cereal (powder to be reconstituted) for baby from 4/6 month",
                "Milk. powder. whol",
                "Instant cereal (powder to be reconstituted) for baby from 6 month",
                "Egg yolk. powd", 
                "Gelatine. dried", 
                "Baby milk. first age. powd",
                "Baby milk. second age. powd",
                "Soya flou", 
                "Sea belt (Saccharina latissima). dried or dehydrated", 
                "Veal stock for sauce and cooking. dehydrated", 
                "Broth. stock or bouillon. meat and vegetables. with fat. dehydrated", 
                "Broth. stock or bouillon. meat and vegetables. defatted. dehydrated",
                "Broth. stock or bouillon. beef. dehydrated",
                "Madeira wine aspic. dehydrated", 
                "Nutritional y", 
                "Chewing gum. without sug", 
                "Chewing gum. sugar level unknown (average)",
                "Baking powder or raising agen", "Prepared mixed meat/fish canned. salad", "Stevia sweeten"]

df_foods_filtered = filter_rows_by_values(df_foods, "Name", remove_foods)

In [8]:
commodities = list(df_foods_filtered["Name"])

In [9]:
data = df_foods_filtered.drop("Name", axis=1).values.tolist()

In [10]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [11]:
# Declare an array to hold our variables. 
foods = [solver.NumVar(0.0, solver.infinity(), item) for item in commodities]

print('Number of variables =', solver.NumVariables())

Number of variables = 3154


In [12]:
# Create the constraints, one per nutrient. (data = nutrients_per_100_gramm)
# gurobipy can express a lists or arrays of constraints with a nicer DSL 
# instead of the many loops necessary with OR-Tools
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver.Constraint(nutrient[1], solver.infinity(), nutrient[0]))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 33


In [13]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i, food in enumerate(foods):
    objective.SetCoefficient(food, 1.0)
objective.SetMinimization()

In [14]:
status = solver.Solve()

# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [15]:
# Display the amounts (in dollars) to purchase of each food.
nutrients_result = [0] * len(nutrients)
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {} gr'.format(commodities[i], food.solution_value()*100))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j] * food.solution_value()
print('\nOptimal daily weight: {:.4f} gr'.format(objective.Value()*100))


Daily Foods:
Cashew nut. dry-grilled. unsalted: 124.74438372656425 gr
Soybean. whole grain: 124.63113840094951 gr
Rusk: 164.4684252875549 gr
Liver. lamb. raw: 0.3099313790025602 gr
Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen: 126.98240822037906 gr
Cod liver oil: 5.705711844442705 gr
Salt. white (sea. igneous or rock). no enrichmen: 1.2223130722566082 gr
Laver (Porphyra sp.). dried or dehydrated: 2.016727705790797 gr

Optimal daily weight: 550.0810 gr


In [16]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Nutrients per day:
Energy (kcal/d): 2500.00 (min 2500.0)
Protein (g/d): 96.00 (min 96.0)
Carbohydrate (g/d): 281.25 (min 281.25)
Fat (g/d): 102.61 (min 97.22222222222224)
Fibres (g/d): 33.41 (min 30.0)
LA (g/d): 23.95 (min 11.11111111111111)
ALA (g/d): 1.94 (min 1.3888888888888888)
EPA (g/d): 0.51 (min 0.125)
DHA (g/d): 0.68 (min 0.125)
Calcium (mg/d): 1048.96 (min 1000.0)
Chloride (mg/d): 3100.00 (min 3100.0)
Copper (mg/d): 4.77 (min 1.6)
Iron (mg/d): 42.72 (min 11.0)
Iodine (µg/d): 165.70 (min 150.0)
Magnesium (mg/d): 774.29 (min 350.0)
Manganese (mg/d): 6.16 (min 3.0)
Phosphorus (mg/d): 1796.19 (min 550.0)
Potassium (mg/d): 3599.23 (min 3500.0)
Selenium (µg/d): 70.00 (min 70.0)
Sodium (mg/d): 2000.00 (min 2000.0)
Zinc (mg/d): 13.97 (min 10.0)
Vitamin D (µg/d): 15.00 (min 15.0)
Vitamin E (mg/d): 18.89 (min 13.0)
Vitamin K1 (µg/d): 101.28 (min 70.0)
Vitamin C (mg/d): 122.16 (min 110.0)
Vitamin B1 or Thiamin (mg/d): 3.53 (min 1.0467000000000002)
Vitamin B2 or Riboflavin (mg/d): 3.41 (

In [17]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)     

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Cashew nut. dry-grilled. unsalted,30.79,22.61,10.42,59.57,21.28,46.46,5.79,2.46,1.84,4.52,0.96,70.56,16.64,0.00,45.11,36.43,36.81,23.57,35.64,0.50,49.99,2.08,3.83,38.55,0.51,12.71,1.79,2.43,9.66,6.51,4.12,0.00,0.11
Soybean. whole grain,20.89,44.79,9.22,23.32,48.50,47.05,82.99,0.00,0.00,26.14,0.00,22.98,45.80,0.45,40.72,45.70,40.66,60.25,15.67,0.19,26.31,0.00,5.61,57.84,6.12,30.68,31.81,7.01,13.39,15.31,52.79,0.00,0.15
Rusk,26.64,16.79,44.79,9.46,15.26,5.84,5.94,3.24,2.42,4.55,47.01,5.51,4.23,19.85,5.10,16.54,9.16,7.31,46.99,41.86,9.30,2.74,7.23,3.61,0.00,5.58,0.48,4.00,6.76,4.09,4.74,0.00,4.83
Liver. lamb. raw,0.02,0.07,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.45,0.05,0.01,0.01,0.01,0.06,0.03,0.22,0.01,0.09,0.01,0.01,0.00,0.01,0.03,0.33,0.19,0.15,0.04,0.09,7.42,0.08
Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen,19.40,15.08,35.49,2.04,12.77,0.00,0.00,0.00,0.00,64.16,28.06,0.00,31.50,3.83,7.79,0.00,12.73,7.87,0.00,31.56,13.63,0.00,73.95,0.00,92.41,50.66,64.45,85.97,70.01,73.72,38.21,73.01,0.00
Cod liver oil,2.05,0.00,0.00,5.56,0.00,0.65,5.27,94.30,95.74,0.01,0.00,0.01,0.01,13.77,0.00,0.00,0.00,0.00,0.00,0.00,0.02,95.10,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,94.84
Salt. white (sea. igneous or rock). no enrichmen,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,23.97,0.04,0.01,0.01,0.00,0.02,0.01,0.01,0.01,23.90,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Laver (Porphyra sp.). dried or dehydrated,0.21,0.66,0.08,0.03,2.19,0.00,0.00,0.00,0.00,0.61,0.00,0.45,1.76,62.07,1.27,1.29,0.58,0.97,1.48,2.00,0.65,0.08,0.31,0.00,0.95,0.33,1.13,0.41,0.04,0.33,0.06,19.56,0.00


In [18]:
foods_df.sum()

Energy (kcal/d)                          100.00
Protein (g/d)                            100.00
Carbohydrate (g/d)                       100.00
Fat (g/d)                                100.00
Fibres (g/d)                             100.00
LA (g/d)                                 100.00
ALA (g/d)                                 99.99
EPA (g/d)                                100.00
DHA (g/d)                                100.00
Calcium (mg/d)                           100.01
Chloride (mg/d)                          100.00
Copper (mg/d)                            100.00
Iron (mg/d)                              100.00
Iodine (µg/d)                             99.99
Magnesium (mg/d)                         100.00
Manganese (mg/d)                          99.99
Phosphorus (mg/d)                        100.01
Potassium (mg/d)                         100.01
Selenium (µg/d)                          100.01
Sodium (mg/d)                            100.02
Zinc (mg/d)                             

In [19]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]).round(2)

# foods_df.loc['total'] = foods_df.iloc[1:, :-1].sum()    
    
display(foods_df)  

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Cashew nut. dry-grilled. unsalted,769.67,21.71,29.31,61.12,7.11,11.13,0.11,0.01,0.01,47.40,29.81,3.37,7.11,0.00,349.28,2.25,661.15,848.26,24.95,9.98,6.99,0.31,0.72,39.04,0.62,0.45,0.06,0.70,1.22,0.21,31.93,0.00,1.93
Soybean. whole grain,522.20,43.00,25.92,23.93,16.20,11.27,1.61,0.00,0.00,274.19,0.00,1.10,19.57,0.75,315.32,2.82,730.34,2168.58,10.97,3.74,3.68,0.00,1.06,58.58,7.48,1.08,1.08,2.02,1.69,0.50,408.79,0.00,2.70
Rusk,666.10,16.12,125.98,9.70,5.10,1.40,0.12,0.02,0.02,47.70,1457.19,0.26,1.81,32.89,39.47,1.02,164.47,263.15,32.89,837.14,1.30,0.41,1.37,3.65,0.00,0.20,0.02,1.15,0.86,0.13,36.68,0.00,87.17
Liver. lamb. raw,0.46,0.07,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.01,0.06,0.00,1.13,0.97,0.15,0.18,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.06,0.02,0.00,0.71,0.30,1.39
Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen,485.07,14.48,99.81,2.10,4.27,0.00,0.00,0.00,0.00,673.01,869.83,0.00,13.46,6.35,60.32,0.00,228.57,283.17,0.00,631.10,1.90,0.00,13.97,0.00,112.89,1.79,2.20,24.76,8.86,2.40,295.87,2.92,0.00
Cod liver oil,51.35,0.00,0.00,5.71,0.00,0.15,0.10,0.48,0.65,0.07,0.00,0.00,0.00,22.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.26,1.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1711.71
Salt. white (sea. igneous or rock). no enrichmen,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.16,743.17,0.00,0.00,0.02,0.04,0.00,0.10,0.21,0.01,477.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Laver (Porphyra sp.). dried or dehydrated,5.14,0.64,0.21,0.03,0.73,0.00,0.00,0.00,0.00,6.41,0.00,0.02,0.75,102.85,9.80,0.08,10.45,34.89,1.03,39.93,0.09,0.01,0.06,0.00,1.16,0.01,0.04,0.12,0.01,0.01,0.44,0.78,0.00


In [20]:
foods_df.sum()

Energy (kcal/d)                          2499.99
Protein (g/d)                              96.02
Carbohydrate (g/d)                        281.24
Fat (g/d)                                 102.61
Fibres (g/d)                               33.41
LA (g/d)                                   23.95
ALA (g/d)                                   1.94
EPA (g/d)                                   0.51
DHA (g/d)                                   0.68
Calcium (mg/d)                           1048.96
Chloride (mg/d)                          3100.00
Copper (mg/d)                               4.77
Iron (mg/d)                                42.72
Iodine (µg/d)                             165.69
Magnesium (mg/d)                          774.29
Manganese (mg/d)                            6.17
Phosphorus (mg/d)                        1796.21
Potassium (mg/d)                         3599.23
Selenium (µg/d)                            70.00
Sodium (mg/d)                            1999.99
Zinc (mg/d)         

In [21]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack (%)': (activities[i] - c.lb())/c.lb()*100} for i, c in enumerate(solver.constraints())]
print(pd.DataFrame(o).round(2))

                                     Name  shadow price  slack (%)
0                         Energy (kcal/d)          0.00       0.00
1                           Protein (g/d)          0.01       0.00
2                      Carbohydrate (g/d)          0.01       0.00
3                               Fat (g/d)          0.00       5.54
4                            Fibres (g/d)          0.00      11.37
5                                LA (g/d)          0.00     115.53
6                               ALA (g/d)          0.00      39.49
7                               EPA (g/d)          0.00     306.10
8                               DHA (g/d)          0.00     443.50
9                          Calcium (mg/d)          0.00       4.90
10                        Chloride (mg/d)          0.00      -0.00
11                          Copper (mg/d)          0.00     198.33
12                            Iron (mg/d)          0.00     288.41
13                          Iodine (µg/d)          0.00      1

In [22]:
# The constraints with a slack value of zero are the most critical for the solution. It these constraints are changed the solution will also change. The most important constraints are
# Energy, Protein, Carbohydrate, Chloride, Sodium, Selenium, Vitamin D, Vitamin B12.

In [38]:
## trying to optimize for calories here.
solver2 = pywraplp.Solver.CreateSolver('GLOP')

In [39]:
# remove foods with no calories
df_foods_filtered2 = df_foods_filtered[df_foods_filtered["Energy (kcal/100g)"] > 0.0]
commodities = list(df_foods_filtered2["Name"])
data = df_foods_filtered2.drop("Name", axis=1).values.tolist()

In [40]:
# Declare an array to hold our variables. 
foods = [solver2.NumVar(0.0, solver2.infinity(), item) for item in commodities]

print('Number of variables =', solver2.NumVariables())

Number of variables = 2173


In [41]:
nutrients[0] = ("Energy (kcal/d)", 0.0)

In [42]:
# Create the constraints, one per nutrient. (data = nutrients_per_100_gramm)
# gurobipy can express a lists or arrays of constraints with a nicer DSL 
# instead of the many loops necessary with OR-Tools
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver2.Constraint(nutrient[1], solver2.infinity(), nutrient[0]))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i])

print('Number of constraints =', solver2.NumConstraints())

Number of constraints = 33


In [43]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver2.Objective()
for i, food in enumerate(foods):
    objective.SetCoefficient(food, data[i][0])
objective.SetMinimization()

In [44]:
status = solver2.Solve()

# Check that the problem has an optimal solution.
if status != solver2.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver2.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [45]:
# Display the amounts (in dollars) to purchase of each food.
nutrients_result = [0] * len(nutrients)
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {} kcal {} gr'.format(commodities[i], food.solution_value()*data[i][0], food.solution_value() * 100))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j] * food.solution_value()
print('\nOptimal daily calories: {:.4f} kcal'.format(objective.Value()))


Daily Foods:
Miso soup. dehydrated. reconstituted: 19.99303403262975 kcal 116.91832767619736 gr
Lamb's lettuce. raw: 76.89731063637166 kcal 457.72208712125985 gr
Spinach. young leaves. raw: 6.162156872851854 kcal 33.67298837623964 gr
Gnocchi. cooked (average): 1458.583383674927 kcal 805.8471733010647 gr
Liver. calf. cooked: 2.0213716192299107 kcal 1.6705550572148022 gr
Liver. turkey. raw: 85.07801636344372 kcal 69.1691189946697 gr
Dry-cured ham. fat and rind removed: 97.6604030115378 kcal 50.86479323517593 gr
Horse mackerel. oily (autumn. winter). raw: 29.456018868913574 kcal 20.038108074090864 gr
Anchovy. fillets. in oil. semi-preserved. drained: 7.759482582810963 kcal 4.263451968577452 gr
Wheat germ oil: 53.574678764646855 kcal 5.952742084960762 gr
Walnut oil: 58.05456190581196 kcal 6.450506878423551 gr
Grapeseed oil: 52.6841448375099 kcal 5.8537938708344335 gr
Cod liver oil: 13.855678715251209 kcal 1.5395198572501343 gr
Parsley. fresh: 19.12507985092427 kcal 44.47692988587039 gr
Se

In [46]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Nutrients per day:
Energy (kcal/d): 1986.08 (min 0.0)
Protein (g/d): 96.00 (min 96.0)
Carbohydrate (g/d): 281.25 (min 281.25)
Fat (g/d): 97.22 (min 97.22222222222224)
Fibres (g/d): 30.00 (min 30.0)
LA (g/d): 11.11 (min 11.11111111111111)
ALA (g/d): 1.39 (min 1.3888888888888888)
EPA (g/d): 0.27 (min 0.125)
DHA (g/d): 0.43 (min 0.125)
Calcium (mg/d): 1149.88 (min 1000.0)
Chloride (mg/d): 3100.00 (min 3100.0)
Copper (mg/d): 1.60 (min 1.6)
Iron (mg/d): 19.80 (min 11.0)
Iodine (µg/d): 2021.79 (min 150.0)
Magnesium (mg/d): 350.00 (min 350.0)
Manganese (mg/d): 3.00 (min 3.0)
Phosphorus (mg/d): 567.60 (min 550.0)
Potassium (mg/d): 3608.33 (min 3500.0)
Selenium (µg/d): 133.14 (min 70.0)
Sodium (mg/d): 5311.28 (min 2000.0)
Zinc (mg/d): 10.40 (min 10.0)
Vitamin D (µg/d): 15.00 (min 15.0)
Vitamin E (mg/d): 13.00 (min 13.0)
Vitamin K1 (µg/d): 807.25 (min 70.0)
Vitamin C (mg/d): 110.00 (min 110.0)
Vitamin B1 or Thiamin (mg/d): 1.05 (min 1.0467000000000002)
Vitamin B2 or Riboflavin (mg/d): 2.20 (min

In [47]:
# Nutrient are far from optimal: Too much Vitamin A, too much Iodine, too much sodium
# It seems to be much better to optimize for weight instead of optimizing for calories

In [48]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)  

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Miso soup. dehydrated. reconstituted,1.01,1.68,0.91,0.60,1.95,0.10,0.82,4.21,2.65,1.83,24.93,4.38,1.12,4.05,2.30,3.12,3.91,1.59,17.56,9.64,1.24,1.95,2.34,0.17,0.00,1.68,0.53,0.70,1.35,0.69,2.43,11.77,0.00
Lamb's lettuce. raw,3.87,9.54,0.81,2.35,35.09,0.41,3.30,16.82,10.58,16.32,6.70,28.61,9.25,4.53,24.85,42.72,24.19,41.86,68.76,0.43,6.16,0.00,7.75,27.73,9.90,18.37,13.73,10.11,19.37,25.85,26.25,0.00,27.03
Spinach. young leaves. raw,0.31,0.72,0.10,0.14,2.69,0.12,4.12,1.24,0.78,3.22,0.76,1.68,3.06,1.00,4.23,6.96,1.90,5.41,5.06,0.16,1.39,0.56,1.45,3.15,1.85,1.32,0.87,0.52,1.14,2.97,3.34,0.00,1.70
Gnocchi. cooked (average),73.44,53.22,97.42,65.32,51.04,0.00,0.00,0.00,0.00,65.60,0.00,0.00,30.52,0.00,39.14,0.00,0.00,35.29,0.00,59.93,61.96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Liver. calf. cooked,0.10,0.33,0.03,0.05,0.00,0.10,0.04,0.00,0.00,0.01,0.00,20.99,0.38,0.00,0.10,0.16,0.89,0.16,0.51,0.03,0.74,0.28,0.08,0.00,0.20,0.24,1.31,1.88,0.51,1.01,1.25,4.47,1.90
Liver. turkey. raw,4.28,13.19,0.00,3.91,0.00,6.97,1.89,2.29,7.19,1.20,0.00,37.18,31.23,0.00,4.74,6.92,34.00,4.10,0.00,1.71,22.40,5.99,1.28,0.00,15.41,13.88,70.75,46.26,73.51,42.32,59.01,69.27,60.27
Dry-cured ham. fat and rind removed,4.92,13.93,0.05,4.97,0.00,0.00,0.00,0.00,0.00,0.00,53.82,0.00,3.60,0.03,0.00,0.00,20.61,0.00,0.00,23.08,0.00,0.00,0.00,0.00,0.00,58.31,6.94,26.42,0.00,17.95,0.00,1.29,0.00
Horse mackerel. oily (autumn. winter). raw,1.48,4.09,0.00,1.58,0.00,0.13,0.61,21.35,32.41,0.10,0.00,1.25,0.86,0.21,1.81,0.67,6.88,2.12,6.44,0.21,0.73,64.79,0.74,0.00,0.00,1.53,1.09,8.02,0.98,4.36,0.00,7.64,0.01
Anchovy. fillets. in oil. semi-preserved. drained,0.39,1.17,0.00,0.37,0.04,0.13,0.16,6.42,5.71,0.70,8.38,0.67,0.69,0.06,0.53,0.14,1.74,0.42,0.26,3.38,1.20,0.48,0.38,0.06,0.02,0.16,0.48,0.99,0.45,0.93,0.08,4.53,0.00
Wheat germ oil,2.70,0.00,0.00,6.12,0.00,25.13,25.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,68.23,0.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [49]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack': (activities[i] - c.lb())} for i, c in enumerate(solver2.constraints())]
print(pd.DataFrame(o).round(2))

                                     Name  shadow price    slack
0                         Energy (kcal/d)          0.00  2500.00
1                           Protein (g/d)          3.50     0.00
2                      Carbohydrate (g/d)          2.61     0.00
3                               Fat (g/d)          8.55     5.39
4                            Fibres (g/d)          1.22     3.41
5                                LA (g/d)          0.69    12.84
6                               ALA (g/d)          0.55     0.55
7                               EPA (g/d)          0.00     0.38
8                               DHA (g/d)          0.00     0.55
9                          Calcium (mg/d)          0.00    48.96
10                        Chloride (mg/d)          0.00    -0.00
11                          Copper (mg/d)          0.16     3.17
12                            Iron (mg/d)          0.00    31.72
13                          Iodine (µg/d)          0.00    15.70
14                       

In [50]:
# The most important constraints are again
# Protein, Carbohydrate, Chloride, Sodium, Selenium, Vitamin D, Vitamin B12.